In [1]:
import pprint
import chessdotcom as cdc
import requests
import pandas as pd
from time import sleep
import sys
import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_data(username,months):
    data = cdc.get_player_game_archives(username).json
    #initialising list to input data
    l=[]
    #from exporation of the pgn text file, we only want the first 21 columns
    col_size=21
    #initialising column names
    col_names=[None]*col_size
    for i in range(1,months+1):
        #grabs the recent month's games
        url = data["archives"][-i]
        games = requests.get(url).json()
        #tells us how many games there are in the month
        total_games=len(games["games"])
        
        for j in range(total_games): 
            #the pgn contains the game data, however it comes as a long string data.
            #lack of delimiter between value and column name, requires to be parsed differently
            game = str(games["games"][j]["pgn"]).replace("[","").replace("]","").split("\n")
            #here we only colect the wanted columns
            chess_data= game[0:col_size]
            
            for k in range(col_size):
                #setting dataframe's columm names
                if col_names[k]==None:
                    col_names[k]=chess_data[k].split(" ")[0]
                #we want to get the data between the ""
                chess_data[k]=chess_data[k].split('"')[1].split('"')[0]
                
            l.append(chess_data)
            
    dataset = pd.DataFrame(l, columns=col_names)
    return dataset

In [3]:
def get_location(username):
    profile=cdc.get_player_profile(username).json
    url=profile["player"]["country"]
    location=requests.get(profile["player"]["country"]).json()
    return location["name"]

In [4]:
#we wish to examine 1 year of chess data
months=12
username="VenDez"
games=get_data(username,months)
games.head()

,Event,Site,Date,Round,White,Black,Result,CurrentPosition,Timezone,ECO,...,UTCDate,UTCTime,WhiteElo,BlackElo,TimeControl,Termination,StartTime,EndDate,EndTime,Link
0,Live Chess,Chess.com,2023.03.01,-,VenDez,waynefoster87,1-0,7Q/1p3p1k/5Bp1/4Pp1p/5P2/6K1/3r2PP/8 b - -,UTC,D06,...,2023.03.01,09:49:11,1359,1400,60,VenDez won by checkmate,09:49:11,2023.03.01,09:51:26,https://www.chess.com/game/live/71403206493
1,Live Chess,Chess.com,2023.03.01,-,VenDez,cparrilli,0-1,rn1qkbnr/ppp1pppp/8/3p4/2PP2b1/8/PP2PPPP/RNBQK...,UTC,D06,...,2023.03.01,10:11:26,1351,1348,60,cparrilli won by resignation,10:11:26,2023.03.01,10:11:32,https://www.chess.com/game/live/71404471019
2,Live Chess,Chess.com,2023.03.01,-,etherman87,VenDez,0-1,r3r1k1/3n1ppp/2p5/p4K2/Pp1Pq3/8/6PP/3R4 w - -,UTC,C01,...,2023.03.01,10:19:18,1294,1358,60,VenDez won by resignation,10:19:18,2023.03.01,10:20:57,https://www.chess.com/game/live/71405014829
3,Live Chess,Chess.com,2023.03.01,-,oliverwakefield,VenDez,1/2-1/2,8/5pp1/p5kp/1q6/2q5/8/5K2/8 b - -,UTC,C00,...,2023.03.01,10:21:27,1393,1359,60,Game drawn by timeout vs insufficient material,10:21:27,2023.03.01,10:23:29,https://www.chess.com/game/live/71405079493
4,Live Chess,Chess.com,2023.03.01,-,VenDez,Yafarov,0-1,2r3k1/5ppp/3P4/8/4P3/1R2B2P/r1p2PP1/3R2K1 w - -,UTC,A40,...,2023.03.01,11:18:36,1351,1375,60,Yafarov won on time,11:18:36,2023.03.01,11:20:44,https://www.chess.com/game/live/71408601109


In [5]:
games["Datetime"]=games["UTCDate"] + " " + games["UTCTime"]

#sipplifying termination status and splitting date into year, month and day
for i in range(games.shape[0]):
    games["Termination"][i]=games["Termination"][i].split()[-1]
    
games.head()

,Event,Site,Date,Round,White,Black,Result,CurrentPosition,Timezone,ECO,...,UTCTime,WhiteElo,BlackElo,TimeControl,Termination,StartTime,EndDate,EndTime,Link,Datetime
0,Live Chess,Chess.com,2023.03.01,-,VenDez,waynefoster87,1-0,7Q/1p3p1k/5Bp1/4Pp1p/5P2/6K1/3r2PP/8 b - -,UTC,D06,...,09:49:11,1359,1400,60,checkmate,09:49:11,2023.03.01,09:51:26,https://www.chess.com/game/live/71403206493,2023.03.01 09:49:11
1,Live Chess,Chess.com,2023.03.01,-,VenDez,cparrilli,0-1,rn1qkbnr/ppp1pppp/8/3p4/2PP2b1/8/PP2PPPP/RNBQK...,UTC,D06,...,10:11:26,1351,1348,60,resignation,10:11:26,2023.03.01,10:11:32,https://www.chess.com/game/live/71404471019,2023.03.01 10:11:26
2,Live Chess,Chess.com,2023.03.01,-,etherman87,VenDez,0-1,r3r1k1/3n1ppp/2p5/p4K2/Pp1Pq3/8/6PP/3R4 w - -,UTC,C01,...,10:19:18,1294,1358,60,resignation,10:19:18,2023.03.01,10:20:57,https://www.chess.com/game/live/71405014829,2023.03.01 10:19:18
3,Live Chess,Chess.com,2023.03.01,-,oliverwakefield,VenDez,1/2-1/2,8/5pp1/p5kp/1q6/2q5/8/5K2/8 b - -,UTC,C00,...,10:21:27,1393,1359,60,material,10:21:27,2023.03.01,10:23:29,https://www.chess.com/game/live/71405079493,2023.03.01 10:21:27
4,Live Chess,Chess.com,2023.03.01,-,VenDez,Yafarov,0-1,2r3k1/5ppp/3P4/8/4P3/1R2B2P/r1p2PP1/3R2K1 w - -,UTC,A40,...,11:18:36,1351,1375,60,time,11:18:36,2023.03.01,11:20:44,https://www.chess.com/game/live/71408601109,2023.03.01 11:18:36


In [6]:
column_names=list(games.columns.values)
print(column_names)

['Event', 'Site', 'Date', 'Round', 'White', 'Black', 'Result', 'CurrentPosition', 'Timezone', 'ECO', 'ECOUrl', 'UTCDate', 'UTCTime', 'WhiteElo', 'BlackElo', 'TimeControl', 'Termination', 'StartTime', 'EndDate', 'EndTime', 'Link', 'Datetime']


In [7]:
features=["White","Black","WhiteElo","BlackElo","Result","Termination","Datetime"]
games=games[features]
games.head()

,White,Black,WhiteElo,BlackElo,Result,Termination,Datetime
0,VenDez,waynefoster87,1359,1400,1-0,checkmate,2023.03.01 09:49:11
1,VenDez,cparrilli,1351,1348,0-1,resignation,2023.03.01 10:11:26
2,etherman87,VenDez,1294,1358,0-1,resignation,2023.03.01 10:19:18
3,oliverwakefield,VenDez,1393,1359,1/2-1/2,material,2023.03.01 10:21:27
4,VenDez,Yafarov,1351,1375,0-1,time,2023.03.01 11:18:36


In [8]:
#getting the right datatypes for each columns
games["Datetime"]=pd.to_datetime(games["Datetime"])
games["WhiteElo"]=pd.to_numeric(games["WhiteElo"])
games["BlackElo"]=pd.to_numeric(games["BlackElo"])
games['Result'] = games['Result'].astype("category")
games['Termination'] = games['Termination'].astype("category")

In [9]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3720 entries, 0 to 3719
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   White        3720 non-null   object        
 1   Black        3720 non-null   object        
 2   WhiteElo     3720 non-null   int64         
 3   BlackElo     3720 non-null   int64         
 4   Result       3720 non-null   category      
 5   Termination  3720 non-null   category      
 6   Datetime     3720 non-null   datetime64[ns]
dtypes: category(2), datetime64[ns](1), int64(2), object(2)
memory usage: 153.2+ KB


In [10]:
#seperating white and black games
white = games.loc[games["White"] == username]
black = games.loc[games["Black"] == username]

#resetting index for location analysis
black.reset_index(drop=True,inplace=True)
white.reset_index(drop=True,inplace=True)

In [11]:
def location_analysis(data):
    data["Location"]=None
    
    for i in range(data.shape[0]):

        if data["White"][0]==username:
            data["Location"][i]=get_location(data["Black"][i])
        else:
            data["Location"][i]=get_location(data["White"][i])
            
        #just to let me know how long is left
        sys.stdout.write('\r')
        sys.stdout.write("progress = " + str(round(i*100/data.shape[0],2)) + "%")
        sys.stdout.flush()
        sleep(0.25)
    return

In [12]:
#optional as the api requests takes really long, but I want to see the locations of whom I played
location_analysis(white)

progress = 99.95%

In [16]:
location_analysis(black)

progress = 99.95%

In [13]:
#cleaning up dataset for white
white=white.drop(["White","Black"], axis=1)
white.rename(columns={'WhiteElo':'Rating','BlackElo':'OppRating'}, inplace = True)
white["Result"]=white["Result"].cat.rename_categories({"1-0":"W", "1/2-1/2": "D","0-1":"L"})
white.head()

,Rating,OppRating,Result,Termination,Datetime,Location
0,1359,1400,W,checkmate,2023-03-01 09:49:11,Scotland
1,1351,1348,L,resignation,2023-03-01 10:11:26,United States
2,1351,1375,L,time,2023-03-01 11:18:36,International
3,1332,1361,L,time,2023-03-01 11:24:40,Singapore
4,1322,1320,L,time,2023-03-01 16:42:07,India


In [15]:
white.sort_values(by=['Datetime'],inplace=True, ascending=False)
white.to_csv("white_games.csv")

In [17]:
#cleaning up dataset for black
black=black.drop(["White","Black"], axis=1)
black.rename(columns={'BlackElo':'Rating','WhiteElo':'OppRating'}, inplace = True)
black["Result"]=black["Result"].cat.rename_categories({"1-0":"L", "1/2-1/2": "D","0-1":"W"})
black.head()

,OppRating,Rating,Result,Termination,Datetime,Location
0,1294,1358,W,resignation,2023-03-01 10:19:18,Singapore
1,1393,1359,D,material,2023-03-01 10:21:27,United Kingdom
2,1330,1342,L,time,2023-03-01 11:20:46,Afghanistan
3,1286,1323,L,time,2023-03-01 11:29:06,Czech Republic
4,1287,1330,W,resignation,2023-03-01 11:31:02,Spain


In [18]:
black.sort_values(by=['Datetime'],inplace=True, ascending=False)
black.to_csv("black_games.csv")

In [19]:
#putting all games together and sorting by most recent games
games=pd.concat([white,black])
games.reset_index(drop=True,inplace=True)
games.sort_values(by=['Datetime'],inplace=True, ascending=False)
games.head()

,Rating,OppRating,Result,Termination,Datetime,Location
1859,1398,1455,L,time,2023-03-13 18:08:54,United States
0,1405,1411,W,time,2023-03-13 17:06:49,United States
1,1396,1354,W,checkmate,2023-03-13 14:14:31,North Macedonia
1860,1389,1415,L,time,2023-03-13 14:01:48,Ukraine
1861,1397,1439,L,time,2023-03-12 19:58:36,United States


In [20]:
games.describe()

,Rating,OppRating
count,3720.000000,3720.000000
mean,1354.294624,1353.113441
std,86.750660,94.718768
min,1155.000000,970.000000
25%,1287.000000,1281.000000
50%,1355.500000,1350.000000
75%,1420.000000,1423.000000
max,1626.000000,1797.000000


In [21]:
#ready to export and perform data_analysis
games.to_csv("chess_games.csv")